In [1]:
import magcolloids as mgc
import os
import sys
import numpy as np
import pandas as pd
from multiprocessing import Pool
import tqdm.auto as tqdm

sys.path.insert(0, '../icenumerics')

import icenumerics as ice
import matplotlib.pyplot as plt

In [2]:
data_directory = "/home/carolina/CairoLattice_ACI_PROJECT/Scripts/Output_Script-15.1"

In [3]:
def do_everything(exp_entry):
    
    e = exp_entry[1].e 
    
    load_obj = mgc.trj_lazyread(os.path.join(data_directory,
                                             "MimicExperimentalData_CairoLattice_25mT-1000s_6x6_EXP%u.lammpstrj"%(e)),
                                                output = ["x","y","z","mux","muy","muz"])    
    bnd = load_obj.get_bounds()
    trj = load_obj.read_trj()
    col_trj = ice.get_ice_trj(trj,bnd, atom_types = 1, trap_types = [2,3])
    
    # Infer Vertices
    frames = col_trj.index.get_level_values("frame").unique()

    v = ice.vertices()
    v.trj_to_vertices(col_trj.loc[frames[::10]])
    
    Cairo = v.vertices
    
    Cairo.index.names = ['frame', 'id']
    
    Cairo.to_csv(os.path.join(data_directory, "MimicExperimentalData_CairoLattice_25mT-1000s_exp%u.dat"%(e)), sep="\t")

In [4]:
e =  np.arange(20,40)
experiments = pd.DataFrame({"e":e.flatten()})
experiments

,e
0,20
1,21
2,22
3,23
4,24
5,25
6,26
7,27
8,28
9,29


In [5]:
if __name__ ==  '__main__': 
    num_processors = 2
    p=Pool(processes = num_processors)
        
    list(tqdm.tqdm(p.imap(do_everything, experiments.iterrows()), total=len(experiments)));